In [1]:
from scrapy import signals
from scrapy.conf import settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import scrapy
import multiprocessing
import datetime

/home/jun/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: ScrapyDeprecationWarning: Module `scrapy.conf` is deprecated, use `crawler.settings` attribute instead
  from ipykernel import kernelapp as app
/home/jun/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: ScrapyDeprecationWarning: Importing from scrapy.xlib.pydispatch is deprecated and will no longer be supported in future Scrapy versions. If you just want to connect signals use the from_crawler class method, otherwise import pydispatch directly if needed. See: https://github.com/scrapy/scrapy/issues/1762


In [2]:
class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(settings)
        #if not hasattr(project, 'crawler'):
        #    self.crawler.install()
        #self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)
        

In [22]:
class CanberraWealtherSpider(scrapy.Spider):
    name = "CanberraWealtherSpider"
    allowed_domains = ["www.bom.gov.au"]
    start_urls = ['http://www.bom.gov.au/act/forecasts/canberra.shtml']

    def parse(self, response):
        forecast_tags=response.xpath('//div[@class="forecast"]')
        for forecast_eachday in forecast_tags:
            Max_Temperature=forecast_eachday.xpath('dl/dd/em[@class="max"]/text()').extract()
            Min_Temperature=forecast_eachday.xpath('dl/dd/em[@class="min"]/text()').extract()
            if Min_Temperature == []:
                Min_Temperature=['Not Exist']
            Summary=forecast_eachday.xpath('dl/dd[@class="summary"]/text()').extract()
            yield{'max':Max_Temperature[0].encode('utf-8'),'min':Min_Temperature[0].encode('utf-8'),'summary':Summary[0].encode('utf-8')}

In [23]:
def main():
    result_queue = Queue()
    crawler = CrawlerWorker(CanberraWealtherSpider(), result_queue)
    crawler.start()
    for item in result_queue.get():
        #print datetime.datetime.now(),item
        print item

In [24]:
if __name__ == '__main__':
    main()

2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Overridden settings: {}
2017-01-13 10:27:05 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.l

{'max': '38', 'min': 'Not Exist', 'summary': 'Hot. A little late rain.'}
{'max': '31', 'min': '19', 'summary': 'Shower or two.'}
{'max': '29', 'min': '15', 'summary': 'Partly cloudy.'}
{'max': '33', 'min': '14', 'summary': 'Mostly sunny.'}
{'max': '36', 'min': '16', 'summary': 'Sunny.'}
{'max': '34', 'min': '22', 'summary': 'Showers.'}
{'max': '31', 'min': '16', 'summary': 'Shower or two.'}
